# CookieCutter
The goal of this project is to predict the calories per serving of a recipe based on the ingredients list.

Training data was scraped from AllRecipes.com

## Import packages

In [ ]:
import numpy as np
import pandas as pd
import re

## Load bag of words vectorization

In [ ]:
import pickle
bow_transformer = pickle.load(open('bow_transformer_2.sav','rb'))
print(len(bow_transformer.vocabulary_)) # Print total number of vocab words
print(bow_transformer.get_feature_names()) # Print all words
ingredient_bow_train = pickle.load(open('ingredient_bow_train_2.sav','rb'))
ingredient_bow_test = pickle.load(open('ingredient_bow_test_2.sav','rb'))

# Modeling

## Import functions

In [ ]:
def modelfit(alg, dtrain, predictors, bow_transformer, performCV=True, useTrainCV=False, printFeatureImportance=True, cv_folds=5, early_stopping_rounds=50):
    """
    Perform cross-validation on training data. Returns model performance and feature importance. 
    """
    from sklearn.ensemble import GradientBoostingRegressor

    #Fit the algorithm on the data
    alg.fit(dtrain, predictors)
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain)
    
    #Perform cross-validation: performCV = True for Random Forest and Gradient Boosting. False for XGBoost.
    if performCV == True:
        from sklearn.model_selection import cross_val_score
        cv_score = cross_val_score(alg, dtrain, predictors, cv=cv_folds, scoring='neg_root_mean_squared_error')
    else:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain, label=predictors)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
        
    #Print model report:
    print ("\nModel Report")
    from scipy import stats
    from sklearn import metrics
    slope, intercept, r_value, p_value, std_err = stats.linregress(dtrain_predictions,predictors)
    print('Slope: ', round(slope,2))
    print('Intercept: ', round(intercept))
    print('Coefficient of Determinant: ', round(r_value**2,2))
    print('p-value: ', p_value)
    print('Standard Error: ', round(std_err,2)) # standard error of the slope
    print('RMSE:', round(np.sqrt(metrics.mean_squared_error(dtrain_predictions, predictors)),2))

    if performCV:
        print ("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
        
    #Print Feature Importance:
    if printFeatureImportance:
        from sklearn.feature_extraction.text import CountVectorizer
        feat_imp = pd.DataFrame(alg.feature_importances_,bow_transformer.get_feature_names(), columns=['coeff'])
        print(feat_imp.sort_values(by='coeff', ascending=False).head())
        import matplotlib.pyplot as plt
        feat_imp['coeff'].sort_values(ascending=False).head().plot(kind='barh', title='Feature Importances').invert_yaxis()
    plt.xlabel('Feature Importance Score')

    
def model_test(alg, bow_train, y_train, bow_test, y_test, linearmodel = False, plot = True, features = True, resid = False, QQ = False):
    """
    Fit model and validate using test data. Returns model performance, feature importance, residual plot, and QQ plot.
    
    If using linear regression, specify linearmodel=True
    """
    alg.fit(bow_train, y_train['totalCal'])
    predictions = pd.DataFrame(y_test['totalCal'])
    predictions['totalCal'] = alg.predict(bow_test)
    predictions['calPerServing'] = predictions['totalCal']/y_test['servings']

    # Model Performance
    from scipy import stats
    from sklearn import metrics
    slope, intercept, r_value, p_value, std_err = stats.linregress(predictions['calPerServing'],y_test['calPerServing'])
    print('Slope: ', round(slope,2))
    print('Intercept: ', round(intercept))
    print('Coefficient of Determinant: ', round(r_value**2,2))
    print('p-value: ', p_value)
    print('Standard Error: ', round(std_err,2)) # standard error of the slope
    print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test['calPerServing'], predictions['calPerServing'])),2))
    
    # Feature Importance
    if linearmodel == True:
        feat_imp = pd.DataFrame(linreg.coef_,bow_transformer.get_feature_names(), columns=['coeff'])
        print(feat_imp.sort_values(by='coeff', ascending=False).head())
    else:
        feat_imp = pd.DataFrame(alg.feature_importances_, bow_transformer.get_feature_names(), columns=['coeff'])
        print('Number of features removed: ', len(feat_imp[feat_imp['coeff']==0])) # number of features removed
        print(feat_imp.sort_values(by='coeff', ascending=False).head())

    # Model visualization
    if plot == True:
        import matplotlib.pyplot as plt
        import seaborn as sns
        fig = plt.figure(figsize=(6,6))

        sns.set_context('poster', font_scale=1)
        ax1 = sns.regplot(y_test['calPerServing'],predictions['calPerServing'])
        ax1.set_xlabel('True Calories per Serving')
        ax1.set_ylabel('Predicted Calories per Serving')

    if features == True:
        fig = plt.figure()
        sns.set_context('poster', font_scale=1)
        ax2 = feat_imp.sort_values(by='coeff', ascending=False).head().plot(kind='barh', title='Feature Importances',legend=False).invert_yaxis()

    if resid == True:
        sns.set_context('notebook', font_scale=1)
        fig = plt.figure()
        ax3 = sns.distplot(y_test['calPerServing']-predictions['calPerServing'])
        ax3.set_xlabel('Residual Calories per Serving', fontsize=20)
        ax3.set_ylabel('Distribution', fontsize=20)
    
    if QQ == True:
        import statsmodels.api as sm
        sns.set_context('notebook', font_scale=1)
        fig = plt.figure()
        ax4 = sm.qqplot(predictions['totalCal'], line='s')
    
    sns.set_context('notebook', font_scale=1)
    return(predictions)

## Load bag of words matrix using pickle

In [ ]:
# Load pickle files
import pickle
bow_transformer = pickle.load(open('bow_transformer_2.sav','rb'))
print(len(bow_transformer.vocabulary_)) # Print total number of vocab words
print(bow_transformer.get_feature_names()) # Print all words
ingredient_bow_train = pickle.load(open('ingredient_bow_train_2.sav','rb'))
ingredient_bow_test = pickle.load(open('ingredient_bow_test_2.sav','rb'))

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
model_test(linreg, ingredient_bow_train, y_train, ingredient_bow_test, y_test, linearmodel=True)

## Random Forest

### RF baseline model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
mid_model = RandomForestRegressor()
modelfit(mid_model, ingredient_bow_train, y_train['totalCal'],bow_transformer)

### Hyperparameter tuning

#### Grid search

In [ ]:
from sklearn.model_selection import GridSearchCV
param_test1 = {'n_estimators':range(10,241,5)}
gsearch1 = GridSearchCV(estimator = RandomForestRegressor(max_features='sqrt', random_state=10), 
param_grid = param_test1, scoring='neg_root_mean_squared_error', n_jobs=4,cv=5)
gsearch1.fit(ingredient_bow_train, y_train['totalCal'])
gsearch1.best_params_, gsearch1.best_score_ # Output

In [ ]:
param_test2 = {'max_depth':range(45,55,1)}
gsearch2 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=195, max_features='sqrt', random_state=10), 
param_grid = param_test2, scoring='neg_root_mean_squared_error', n_jobs=4,cv=5)
gsearch2.fit(ingredient_bow_train, y_train['totalCal'])
gsearch2.best_params_, gsearch2.best_score_ # Output

In [ ]:
param_test3 = {'min_samples_split':range(1,20,1)}
gsearch3 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=195, max_depth=51, max_features='sqrt', random_state=10), 
param_grid = param_test3, scoring='neg_root_mean_squared_error', n_jobs=4,cv=5)
gsearch3.fit(ingredient_bow_train, y_train['totalCal'])
gsearch3.best_params_, gsearch3.best_score_ # Output

In [ ]:
param_test4 = {'min_samples_leaf':range(1,15,1)}
gsearch4 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=195, max_depth=51, min_samples_split=2, max_features='sqrt', random_state=10), 
param_grid = param_test4, scoring='neg_root_mean_squared_error', n_jobs=4,cv=5)
gsearch4.fit(ingredient_bow_train, y_train['totalCal'])
gsearch4.best_params_, gsearch4.best_score_ # Output

#### Randomized grid search
This takes ~ 10 minutes.

In [ ]:
grid_param = {'n_estimators': [50, 100, 200, 400, 800] , #number of trees
             'max_features': ['auto','sqrt','log2'], # max number of features to consider at every split
             'max_depth': [10, 20, 30, 40, 50], # max levels in a tree
             'min_samples_split': [2,5,10,15,20], # min number of samples required to split a node
             'min_samples_leaf': [1,2,5,10,15] # min number of samples required at each leaf node
             }

from sklearn.model_selection import RandomizedSearchCV
RFR = RandomForestRegressor(random_state=10)
RFR_random = RandomizedSearchCV(estimator=RFR, param_distributions = grid_param, n_iter=500, cv=5, 
                                verbose=2, random_state=42, n_jobs=-1)
RFR_random.fit(ingredient_bow_train, y_train['totalCal'])
RFR_random.best_params_, RFR_random.best_score_ # Output

### Final model prediction

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor(n_estimators=195, min_samples_split=2, min_samples_leaf=1, 
                                  max_features='sqrt', max_depth=51, random_state=10)

model_test(RF_model, ingredient_bow_train, y_train, ingredient_bow_test, y_test)

##  Gradient Boosting Regressor

### GBR baseline model

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
mid_model = GradientBoostingRegressor(loss="ls")
modelfit(mid_model, ingredient_bow_train, y_train['totalCal'],bow_transformer)

### Hyperparameter tuning

#### Fix learning rate & Number of estimators for tuning tree-based parameters
`min_samples_split` should be 0.5-1% of the total dataset
`min_samples_leaf`should be ~1/10th of `min_samples_split`
`learning rate` standard is 0.1. Can go up to 0.3.
`n_estimators` should be < 100.

If optimal estimators is around 20, lower learning rate to 0.05 and rerun grid search.
If optimal estimators is too high (~100), increase learning rate. This will cause tuning of other parameters to take a long time.

In [ ]:
from sklearn.model_selection import GridSearchCV
param_test1 = {'n_estimators':range(20,241,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.4, min_samples_split=20, min_samples_leaf=2, max_depth=8, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test1, scoring='neg_root_mean_squared_error', n_jobs=4,cv=5)
gsearch1.fit(ingredient_bow_train, y_train['totalCal'])
gsearch1.best_params_, gsearch1.best_score_ # Output

In [ ]:
gsearch1.cv_results_

#### Tune tree parameters

In [ ]:
param_test2 = {'max_depth':range(5,16,2), 'min_samples_split':range(200,2001,200)}
gsearch2 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.4, n_estimators=60, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2, scoring='neg_root_mean_squared_error', n_jobs=4, cv=5)
gsearch2.fit(ingredient_bow_train, y_train['totalCal'])
gsearch2.best_params_, gsearch2.best_score_

In [ ]:
param_test3 = {'min_samples_split':range(900,1200,10), 'min_samples_leaf':range(2,82,4)}
gsearch3 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.4, n_estimators=60, max_depth=7, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test3, scoring='neg_root_mean_squared_error', n_jobs=4, cv=5)
gsearch3.fit(ingredient_bow_train, y_train['totalCal'])
gsearch3.best_params_, gsearch3.best_score_

In [ ]:
param_test4 = {'max_features':range(7,30,2)}
gsearch4 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.4, n_estimators=60, max_depth=7, min_samples_split=930, min_samples_leaf=6, subsample=0.8, random_state=10),
param_grid = param_test4, scoring='neg_root_mean_squared_error', n_jobs=4, cv=5)
gsearch4.fit(ingredient_bow_train, y_train['totalCal'])
gsearch4.best_params_, gsearch4.best_score_

In [ ]:
modelfit(gsearch4.best_estimator_, ingredient_bow_train, y_train['totalCal'],bow_transformer)

#### Tune subsample

In [ ]:
param_test5 = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9]}
gsearch5 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.4, n_estimators=60, max_depth=7, min_samples_split=930, min_samples_leaf=6, subsample=0.8, max_features=27, random_state=10),
param_grid = param_test5, scoring='neg_root_mean_squared_error', n_jobs=4, cv=5)
gsearch5.fit(ingredient_bow_train, y_train['totalCal'])
gsearch5.best_params_, gsearch5.best_score_

#### Tune learning rate and number of trees

In [ ]:
# 1/2 learning rate with 2X trees (n_estimators)
gbm_tuned_1 = GradientBoostingRegressor(learning_rate=0.2, n_estimators=120, max_depth=7, min_samples_split=930, min_samples_leaf=6, subsample=0.85, max_features=27, random_state=10)
modelfit(gbm_tuned_1, ingredient_bow_train, y_train['totalCal'],bow_transformer)

In [ ]:
# 1/10 learning rate with 10X trees (n_estimators)
gbm_tuned_2 = GradientBoostingRegressor(learning_rate=0.04, n_estimators=600, max_depth=7, min_samples_split=930, min_samples_leaf=6, subsample=0.85, max_features=27, random_state=10)
modelfit(gbm_tuned_2, ingredient_bow_train, y_train['totalCal'],bow_transformer)

In [ ]:
# 1/15 learning rate with 15X trees (n_estimators)
gbm_tuned_3 = GradientBoostingRegressor(learning_rate=0.03, n_estimators=900, max_depth=7, min_samples_split=950, min_samples_leaf=6, subsample=0.8, max_features=21, random_state=10)
modelfit(gbm_tuned_3, ingredient_bow_train, y_train['totalCal'],bow_transformer)

### Final model prediction

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
GBR_model = GradientBoostingRegressor(loss="ls", learning_rate=0.04, n_estimators=600, max_depth=7, min_samples_split=930, min_samples_leaf=6, subsample=0.85, max_features=27, random_state=10)
model_test(GBR_model, ingredient_bow_train, y_train, ingredient_bow_test, y_test)

## XGBoost

### XGBoost baseline model

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

XGB_model = XGBRegressor()
modelfit(XGB_model, ingredient_bow_train, y_train['totalCal'],bow_transformer, performCV=False)

### Hyperparameter tuning

#### Tune max depth & min child weight
These parameters have the highst impact on model outcome

In [ ]:
from sklearn.model_selection import GridSearchCV

param_test1 = {
 'max_depth':range(5,8,1),
 'min_child_weight':range(5,11,1)
}

gsearch1 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='neg_root_mean_squared_error',n_jobs=4, cv=5)

gsearch1.fit(ingredient_bow_train, y_train['totalCal'])
gsearch1.best_params_, gsearch1.best_score_

In [ ]:
gsearch1.cv_results_

#### Tune gamma

In [ ]:
param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}

gsearch2 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=140, max_depth=6,
 min_child_weight=9, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test2, scoring='neg_root_mean_squared_error',n_jobs=4, cv=5)

gsearch2.fit(ingredient_bow_train, y_train['totalCal'])
gsearch2.best_params_, gsearch1.best_score_

#### Tune subsample and colsample_bytree

In [ ]:
param_test3 = {
 'subsample':[i/100.0 for i in range(60,90,5)],
 'colsample_bytree':[i/100.0 for i in range(70,90,5)]
}

gsearch3 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=140, max_depth=6,
 min_child_weight=9, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test3, scoring='neg_root_mean_squared_error',n_jobs=4, cv=5)

gsearch3.fit(ingredient_bow_train, y_train['totalCal'])
gsearch3.best_params_, gsearch1.best_score_

#### Tune regularization parameters
This will reduce overfitting

In [ ]:
param_test4 = {
 'reg_alpha':[1e-8, 1e-7, 1e-6, 1e-5, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 100]
}

gsearch4 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=140, max_depth=6,
 min_child_weight=9, gamma=0, subsample=0.65, colsample_bytree=0.7,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test4, scoring='neg_root_mean_squared_error',n_jobs=4, cv=5)

gsearch4.fit(ingredient_bow_train, y_train['totalCal'])
gsearch4.best_params_, gsearch1.best_score_

Best parameters: 
* max_depth = 6
* min_child_weight = 9
* gamma = 0
* colsample_bytree=0.8
* subsample = 0.8

In [ ]:
model3 = XGBRegressor( learning_rate =0.05, n_estimators=280, max_depth=6,
 min_child_weight=9, gamma=0, subsample=0.65, colsample_bytree=0.7,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1, reg_alpha=1e-6, seed=27)

modelfit(model3, ingredient_bow_train, y_train['totalCal'],bow_transformer, performCV=False)

### Final model prediction

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

xgb_model = XGBRegressor(learning_rate =0.05, n_estimators=280, max_depth=6,
     min_child_weight=9, gamma=0, subsample=0.65, colsample_bytree=0.7,
     objective= 'reg:squarederror', nthread=4, scale_pos_weight=1, reg_alpha=1e-6, seed=27)

model_test(xgb_model, ingredient_bow_train, y_train, ingredient_bow_test, y_test)